## Initialization

In [ ]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import calendar
import datetime
from datetime import datetime, timedelta, date
import warnings
import os

#settings
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 1200)
pd.set_option('display.width', 1200)
warnings.filterwarnings("ignore")
%matplotlib inline

#variables
colour=['maroon','r','g','darkgreen','c','teal','b','navy','indigo','m','deeppink','orange','sienna','yellow','khaki','olive','tan','black','grey','brown']

print("Packages installed")



Packages installed


In [ ]:
def csv_to_df(filename):
    file_path = f"/work/{filename}.csv"
    df = pd.read_csv(file_path, sep=",", encoding="UTF-8")
    return df

def df_to_csv(df, filename):
    df.to_csv(f"{filename}.csv", index=False)
    print(f"{filename}.csv er laget")
    return



## Read Clean data, drop some columns

In [ ]:
df = csv_to_df("cleaned_data_03_12")

#,player_id,country_of_birth,country_of_citizenship,date_of_birth,height_in_cm,sub_position,club_id,year,goals_for,goals_against,goals,assists,red_cards,yellow_cards,minutes_played,market_value_in_eur,date,age_at_evaluation,domestic_competition_id,log_market_value_base10


df.drop(["player_id", "country_of_birth", "date_of_birth", "date", "market_value_in_eur"], axis=1, inplace=True)

df.head()

,country_of_citizenship,height_in_cm,sub_position,club_id,year,goals_for,goals_against,goals,assists,red_cards,yellow_cards,minutes_played,age_at_evaluation,domestic_competition_id,log_market_value_base10
0,Germany,184.0,Centre-Forward,398.0,2012.75,31.0,14.0,11.0,1.0,0.0,6.0,1483.0,35,IT1,6.60206
1,Germany,184.0,Centre-Forward,398.0,2013.25,24.0,15.0,5.0,2.0,0.0,2.0,1102.0,35,IT1,6.30103
2,Germany,184.0,Centre-Forward,398.0,2013.75,14.0,21.0,4.0,2.0,0.0,1.0,950.0,36,IT1,6.00000
3,Germany,184.0,Centre-Forward,398.0,2014.25,24.0,22.0,4.0,3.0,0.0,1.0,1270.0,36,IT1,6.00000
4,Germany,184.0,Centre-Forward,398.0,2014.75,31.0,18.0,4.0,3.0,0.0,2.0,496.0,37,IT1,6.00000


## TargetEncode: Country, Position, Club, League

In [ ]:
from category_encoders.target_encoder import TargetEncoder

# Initialize TargetEncoder
encoder = TargetEncoder()

# Specify columns to target encode
columns_to_encode = ['country_of_citizenship', 'sub_position', 'club_id', 'domestic_competition_id']

df['club_id'] = df['club_id'].astype(str)

# Fit and transform the DataFrame with target encoding
df_encoded = encoder.fit_transform(df[columns_to_encode], df['log_market_value_base10'])

# Merge the encoded DataFrame with the original DataFrame
df = pd.concat([df, df_encoded.add_suffix('_encoded')], axis=1)
df = df.drop(["country_of_citizenship", "sub_position", "domestic_competition_id", "club_id"], axis=1)
# Display the resulting DataFrame
print(df)
df_to_csv(df, "encoded_data_03_12")


        height_in_cm     year  goals_for  goals_against  goals  assists  red_cards  yellow_cards  minutes_played  age_at_evaluation  log_market_value_base10  country_of_citizenship_encoded  sub_position_encoded  club_id_encoded  domestic_competition_id_encoded
0              184.0  2012.75       31.0           14.0   11.0      1.0        0.0           6.0          1483.0                 35                 6.602060                        6.445992              6.241155         6.659857                         6.525779
1              184.0  2013.25       24.0           15.0    5.0      2.0        0.0           2.0          1102.0                 35                 6.301030                        6.445992              6.241155         6.659857                         6.525779
2              184.0  2013.75       14.0           21.0    4.0      2.0        0.0           1.0           950.0                 36                 6.000000                        6.445992              6.241155       

In [ ]:
from category_encoders.target_encoder import TargetEncoder

# Initialize TargetEncoder
encoder = TargetEncoder()
df = pd.get_dummies(df, columns = ['sub_position'])

print(df.head())

# Identify columns starting with 'sub_position_'
columns_to_convert_to_int = [col for col in df.columns if col.startswith('sub_position_')]

# Convert selected columns to integers
df[columns_to_convert_to_int] = df[columns_to_convert_to_int].astype(int)


# Specify columns to target encode
columns_to_encode = ['country_of_citizenship', 'club_id', 'domestic_competition_id']

df['club_id'] = df['club_id'].astype(str)

# Fit and transform the DataFrame with target encoding
df_encoded = encoder.fit_transform(df[columns_to_encode], df['log_market_value_base10'])

# Merge the encoded DataFrame with the original DataFrame
df = pd.concat([df, df_encoded.add_suffix('_encoded')], axis=1)
df = df.drop(["country_of_citizenship", "domestic_competition_id", "club_id"], axis=1)
# Display the resulting DataFrame
print(df)
df_to_csv(df, "encoded_data_03_15")


  country_of_citizenship  height_in_cm  club_id     year  goals_for  goals_against  goals  assists  red_cards  yellow_cards  minutes_played  age_at_evaluation domestic_competition_id  log_market_value_base10  sub_position_Attacking Midfield  sub_position_Central Midfield  sub_position_Centre-Back  sub_position_Centre-Forward  sub_position_Defensive Midfield  sub_position_Goalkeeper  sub_position_Left Midfield  sub_position_Left Winger  sub_position_Left-Back  sub_position_Right Midfield  sub_position_Right Winger  sub_position_Right-Back  sub_position_Second Striker
0                Germany         184.0    398.0  2012.75       31.0           14.0   11.0      1.0        0.0           6.0          1483.0                 35                     IT1                  6.60206                            False                          False                     False                         True                            False                    False                       False               

In [ ]:

df['club_id'] = df['club_id'].astype(str)
df_encoded = pd.get_dummies(df, columns=['sub_position', 'country_of_citizenship', 'club_id', 'domestic_competition_id'])

# Identify columns resulting from one-hot encoding
one_hot_encoded_columns = [col for col in df_encoded.columns if col.startswith('sub_position_') 
                                                   or col.startswith('country_of_citizenship_')
                                                   or col.startswith('club_id_')
                                                   or col.startswith('domestic_competition_id_')]

# Convert only one-hot encoded columns to integers
df_encoded[one_hot_encoded_columns] = df_encoded[one_hot_encoded_columns].astype(int)

# Print the resulting DataFrame (optional)
print(df_encoded.head())

# Merge the encoded DataFrame with the original DataFrame
#df = pd.concat([df, df_encoded.add_suffix('_encoded')], axis=1)
#df = df.drop(["country_of_citizenship", "domestic_competition_id", "club_id"], axis=1)
# Display the resulting DataFrame
df_to_csv(df_encoded, "encoded_data_03_18")


KernelInterrupted: Execution interrupted by the Jupyter kernel.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=039d1f4a-fb47-4bbf-8592-3573ff918cac' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>